In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv('C:\\Users\\KAJAL\\Desktop\\symptomAI\\application\\Datasets\\dataset.csv')
precaution_df = pd.read_csv('C:\\Users\\KAJAL\\Desktop\\symptomAI\\application\\Datasets\\symptom\\symptom_precaution.csv')

df_with_precautions = df.merge(precaution_df, on='Disease')

df = pd.read_csv('C:\\Users\\KAJAL\\Desktop\\symptomAI\\application\\Datasets\\dataset.csv')
df.head()
df1 = pd.read_csv('C:\\Users\\KAJAL\\Desktop\\symptomAI\\application\\Datasets\\symptom\\Symptom_severity.csv')
df1.head()
df2 = pd.read_csv('C:\\Users\\KAJAL\\Desktop\\symptomAI\\application\\Datasets\\symptom\\symptom_Description.csv')
df2.head()
df3 = pd.read_csv('C:\\Users\\KAJAL\\Desktop\\symptomAI\\application\\Datasets\\symptom\\symptom_precaution.csv')
df3.head()

df.isna().sum()
df.isnull().sum()
cols = df.columns
data = df[cols].values.flatten()
s = pd.Series(data)
s = s.str.strip()
s = s.values.reshape(df.shape)
df = pd.DataFrame(s, columns=df.columns)
df = df.fillna(0)
df.head()

vals = df.values
symptoms = df1['Symptom'].unique()
for i in range(len(symptoms)):
    vals[vals == symptoms[i]] = df1[df1['Symptom'] == symptoms[i]]['weight'].values[0]
d = pd.DataFrame(vals, columns=cols)
d = d.replace('dischromic _patches', 0)
d = d.replace('spotting_ urination',0)
df = d.replace('foul_smell_of urine',0)
df.head()

(df[cols] == 0).all()
df['Disease'].value_counts()
df['Disease'].unique()
data = df.iloc[:,1:].values
labels = df['Disease'].values

In [5]:
x_train_disease, x_test_disease, y_train_disease, y_test_disease = train_test_split(data, labels, test_size=0.2, random_state=42)
print(x_train_disease.shape)
print(y_train_disease.shape)
print(x_test_disease.shape)
print(y_test_disease.shape)
print(data)
label_encoder = LabelEncoder()
y_train_disease = label_encoder.fit_transform(y_train_disease)
y_test_disease = label_encoder.transform(y_test_disease)

(3936, 17)
(3936,)
(984, 17)
(984,)
[[1 3 4 ... 0 0 0]
 [3 4 0 ... 0 0 0]
 [1 4 0 ... 0 0 0]
 ...
 [6 4 0 ... 0 0 0]
 [3 3 3 ... 0 0 0]
 [3 7 4 ... 0 0 0]]


In [6]:
def train_and_evaluate_disease_model(model, x_train, y_train, x_test, y_test):

    model.fit(x_train, y_train)

    y_pred = model.predict(x_test)

    f1 = f1_score(y_test, y_pred, average='macro')

    accuracy = accuracy_score(y_test, y_pred)

    confusion_mat = confusion_matrix(y_test, y_pred)

    return f1, accuracy, confusion_mat

svm_model_disease = SVC(kernel='linear')
f1_svm_disease, accuracy_svm_disease, confusion_mat_svm_disease = train_and_evaluate_disease_model(svm_model_disease, x_train_disease, y_train_disease, x_test_disease, y_test_disease)

In [7]:
def predict_disease_and_precautions(symptoms):
    disease_prediction = svm_model_disease.predict([symptoms])

    #precaution_prediction = svm_model_precaution.predict([symptoms])
    return [disease_prediction]

symptoms = [3, 5, 3, 5, 4, 4, 3, 2, 3, 0, 0, 0, 0, 0, 0, 0, 0]
disease = predict_disease_and_precautions(symptoms)

disease_name = label_encoder.inverse_transform(disease)
print("Predicted Disease:", disease_name)


Predicted Disease: ['Migraine']


C:\Users\KAJAL\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:153: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [8]:
from sklearn.metrics import accuracy_score

y_pred_disease = svm_model_disease.predict(x_test_disease)

accuracy = accuracy_score(y_test_disease, y_pred_disease)
print(f"Accuracy:{accuracy * 100:.2f}%")

Accuracy:98.17%


In [11]:
import joblib

joblib.dump(svm_model_disease,"disease_symptom.joblib")

['disease_symptom.joblib']